# Approximate $\pi$ via Metropolis MC

This file contains the implementation of a Monte Carlo-Simulation of Pi, using a Metropolis Markov Chain algorithm. For automated testing I use ipython_nose which can be found [in this github-repo](https://github.com/taavi/ipython_nose). To use cell magic you need to install [this pull request](https://github.com/taavi/ipython_nose/pull/11). If you execute a cell containing %nose then all functions with a name starting with test will be executed. If you use %%nose only tests in the same cell are being executed.


In [1]:
import numpy as np
import numpy.testing as nptest

## Create a Markov-Chain:


In [2]:
def create_trial_chain(N,max_step_size):
    if(type(N) == int and N > 0):#Checking if first argument is valid
        if((type(max_step_size) == float or type(max_step_size)== int) # Checking if second argument is valid
           and max_step_size>=0 and max_step_size <=1):
            # initialize with -1 to make shure that all values will be changed
            markov_chain = np.ones((N,2))*(-1) 
            #set a random start_point
            markov_chain[0,:]=np.random.rand(2)

            for i in range(1,N):
                previous = markov_chain[i-1,:]
                delta = (np.random.rand(2)-0.5)*max_step_size*2 
                # if max_step_size is 1 this is a random number from [-1,1]
                proposal = previous+delta
                if ((proposal <= 1)*(proposal >= 0)).all():
                    markov_chain[i,:]=proposal
                else:
                    markov_chain[i,:]=previous
                    
            return markov_chain
        else:
            raise TypeError("max_step_size is not a float between 0 and 1")
    else:
        raise TypeError("N is not a positive integer")

In [3]:
def approx_pi_markov(N,max_step_size):
    trial_chain = create_trial_chain(N,max_step_size)
    return 4/N *np.sum(np.power(trial_chain[:,0],2) + np.power(trial_chain[:,1],2) <= 1)

## Tests


In [4]:
%load_ext ipython_nose

In [6]:
%%nose
def test_create_trial_chain_returns_nx2_array_with_positive_input():
    nptest.assert_equal(np.shape(create_trial_chain(3,1)),(3,2))
    
def test_do_not_create_trial_chain_with_0_input():
    with nptest.assert_raises(TypeError):
        create_trial_chain(0,1)
    
def test_do_not_create_trial_chain_with_negative_input():
    with nptest.assert_raises(TypeError):
        create_trial_chain(-3,1)
    
def test_do_not_create_trial_chain_with_float():
    with nptest.assert_raises(TypeError):
        create_trial_chain(3.3,1)
        
def test_do_not_create_trial_chain_with_string_as_second_arg():
    with nptest.assert_raises(TypeError):
        create_trial_chain(3,'foo')
        
def test_do_not_create_trial_chain_with_second_arg_larger_one():
    with nptest.assert_raises(TypeError):
        create_trial_chain(3,1.4)

def test_do_not_create_trial_chain_with_second_arg_smaller_0():
    with nptest.assert_raises(TypeError):
        create_trial_chain(3,-1)

def test_create_trial_chain_returns_numbers_between_0_and_1():
    for i in range(50):
        test_array = create_trial_chain(1000,0.5)
        assert ((test_array >=0) * (test_array <= 1)).all()

def test_do_not_app_pi_with_second_arg_smaller_0():
    with nptest.assert_raises(TypeError):
        approx_pi_markov(10000,-1)
        

def test_approx_pi_markov_is_close_to_py():
    for i in range(20):
        assert abs(approx_pi_markov(10000,0.5)-np.pi) <= 0.1

10/10 tests passed